# Objetivo

Avaliar se há clusters no portfólio 1. Em um segundo momento, a partir de uma clusterização global (na base toda), buscar novos clientes na base completa.

# Método

A partir do conjunto reduzido de features (47), aplicar um processo de clusterização (AgglomerativeClustering).

# Leitura dos arquivos

In [44]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import feather

## Leitura da lista de features a excluir

In [105]:
# obtida em EDA Pandas_profiling_sample_10k_market.ipynb
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)

133

## Carregamento Portfólio 1

In [106]:
p1 = pd.read_csv('clientes2_merge.csv')
p1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Columns: 182 entries, Unnamed: 0 to qt_filiais
dtypes: bool(16), float64(144), int64(2), object(20)
memory usage: 743.0+ KB


## Redução no número de features

In [107]:
p1.drop(columns=exclude_variables, inplace= True)
columns_to_drop = ['Unnamed: 0', 'id','fl_epp','qt_filiais', 'qt_socios_masculino','qt_socios_pj','vl_total_veiculos_leves_grupo','vl_total_veiculos_pesados_grupo']
p1.drop(columns= columns_to_drop ,inplace=True)
p1.shape

(566, 41)

# PCA

## Dados do portfólio

Para usar nos dados do portfólio, temos que passar as strings para valor numérico.

In [137]:
p1micro.dtypes==object

fl_matriz                                False
de_natureza_juridica                      True
sg_uf                                     True
natureza_juridica_macro                   True
de_ramo                                   True
setor                                     True
idade_empresa_anos                       False
idade_emp_cat                             True
fl_me                                    False
fl_sa                                    False
fl_epp                                   False
fl_mei                                   False
fl_ltda                                  False
dt_situacao                               True
fl_st_especial                           False
fl_email                                 False
fl_telefone                              False
fl_rm                                     True
nm_divisao                                True
nm_segmento                               True
fl_spa                                   False
fl_antt      

In [128]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [138]:
p1enc=pd.DataFrame()
categorical_cols = p1micro.columns
p1enc = p1micro[categorical_cols]
p1enc.columns

#p1enc = p1enc[categorical_cols].apply(lambda col: le.fit_transform(col))

Index(['fl_matriz', 'de_natureza_juridica', 'sg_uf', 'natureza_juridica_macro',
       'de_ramo', 'setor', 'idade_empresa_anos', 'idade_emp_cat', 'fl_me',
       'fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda', 'dt_situacao', 'fl_st_especial',
       'fl_email', 'fl_telefone', 'fl_rm', 'nm_divisao', 'nm_segmento',
       'fl_spa', 'fl_antt', 'fl_veiculo', 'fl_optante_simples',
       'vl_total_veiculos_pesados_grupo', 'vl_total_veiculos_leves_grupo',
       'fl_optante_simei', 'sg_uf_matriz', 'de_saude_tributaria',
       'de_saude_rescencia', 'nu_meses_rescencia', 'de_nivel_atividade',
       'fl_simples_irregular', 'empsetorcensitariofaixarendapopulacao',
       'nm_meso_regiao', 'nm_micro_regiao', 'fl_passivel_iss', 'qt_socios',
       'qt_socios_pj', 'idade_media_socios', 'qt_socios_masculino',
       'qt_socios_feminino', 'de_faixa_faturamento_estimado',
       'de_faixa_faturamento_estimado_grupo', 'vl_faturamento_estimado_aux',
       'vl_faturamento_estimado_grupo_aux', 'qt_filiais'],
 

In [136]:
p1enc.empsetorcensitariofaixarendapopulacao.dtype

dtype('O')

In [133]:
for feature in p1enc:
    if p1enc[feature].dtype == object:
        print(feature, p1enc[feature].dtype)
        p1enc.fillna('other',inplace = True)        
        p1enc[feature] = le.fit_transform(p1enc[feature])

empsetorcensitariofaixarendapopulacao object


TypeError: argument must be a string or number

In [101]:
p1enc = p1enc.apply(lambda col: le.fit_transform(col))

TypeError: ('argument must be a string or number', 'occurred at index nm_meso_regiao')

In [40]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
p1enc['cluster'] = cluster.fit_predict(p1enc)

/home/bruno_dbki/codenation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [41]:
# aqui, estou somando um valor aleatório para fins de visualização.
# Se não for feito, um grupo grande de pontos fica condensado em um único ponto no gráfico
# e perde-se a noção de cluster
p1enc[p1enc.columns[0]]=p1enc[p1enc.columns[0]] + ((np.random.rand(p1enc.shape[0])-0.5) *0.5)
p1enc[p1enc.columns[1]]=p1enc[p1enc.columns[1]] + ((np.random.rand(p1enc.shape[0])-0.5) *0.5)
p1enc[p1enc.columns[2]]=p1enc[p1enc.columns[2]] + ((np.random.rand(p1enc.shape[0])-0.5) *0.5)
p1enc.head(5)

/home/bruno_dbki/codenation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/bruno_dbki/codenation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/bruno_dbki/codenation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://p

,de_nivel_atividade,de_saude_tributaria,fl_mei,cluster
0,0.018078,2.900141,-0.180499,2
1,-0.107714,3.026864,-0.029740,2
2,-0.105014,2.912996,-0.134480,2
3,-0.087917,3.229180,0.219290,2
4,-0.225257,2.882307,0.039460,2


In [42]:
import plotly.express as px
fig = px.scatter_3d(p1enc, x=p1enc.columns[0], y= p1enc.columns[1], z = p1enc.columns[2],color = 'cluster')
fig.show()

## Conclusão até o momento

Clusterização parece interessante. Observar que, das três categorias avaliadas, o `de_ramo` não teve muita influência, porque está bem polarizado em uma das classes. Próximos passos: incluir mais features (sabendo que vai impedir a visualização em mais de 3 dimensões) e aplicar redução de dimensionalidade (PCA? antes ou depois da clusterização?).